### Start loading data into gold layer


In [0]:
%run /Workspace/Users/mbothe7@hotmail.com/databricks-end-to-end-project/01-initial-config

In [0]:
class goldLoader():
    def __init__(self,env):
      Conf = InitialConfig()
      self.catalog = env
      self.gold_path=Conf.gold_path

    def read_silver_traffic(self):
      print("Reading silver traffic")
      df = (spark.readStream
            .table(self.catalog + ".silver.traffic_data")
          )
      print("Silver traffic read complete")
      return df

    def read_silver_roads(self):
      print("Reading silver roads")
      df = (spark.readStream
            .table(self.catalog + ".silver.roads_data")
          )
      print("Silver roads read complete")
      return df

    def create_vehicle_intensity(self,df):
      from pyspark.sql.functions import col
      print('Creating Vehicle Intensity column : ',end='')
      df_in = df.withColumn('Vehicle_Intensity',col('motor_count_add') / col('Link_length_km') )

      print(f"added column to ",self.catalog + ".gold.vehicle_intensity")
      return df_in

    def create_load_time(self,df):
      from pyspark.sql.functions import current_timestamp
      print('Creating Load Time column : ',end='')
      df_timestamp = df.withColumn('Load_Time', current_timestamp() )
      print(f"adding load time column complete")
      return df_timestamp
    
    def write_gold_traffic(self,df):
      print("Writing gold traffic")
      gold_traffic= (
        df.writeStream
          .format("delta")
          .option("checkpointLocation", self.gold_path + "/traffic_checkpoint/checkpt")
          .trigger(availableNow=True)
          .outputMode("append")
          .queryName(self.catalog + ".gold.traffic_data")
          .toTable(self.catalog + ".gold.traffic_data")
        )
      gold_traffic.awaitTermination()
      print("Gold traffic write complete")
      
  
    def write_gold_roads(self,df):
      print("Writing gold roads")
      gold_roads= (
        df.writeStream
          .format("delta")
          .option("checkpointLocation", self.gold_path + "/gold_checkpoint/checkpt")
          .trigger(availableNow=True)
          .outputMode("append")
          .queryName(self.catalog + ".gold.roads_data")
          .toTable(self.catalog + ".gold.roads_data")
      )
      gold_roads.awaitTermination()
      print("Gold traffic write complete")


    def load_gold_traffic(self):
      print("Loading gold traffic")
      df = self.read_silver_traffic()
      df = self.create_vehicle_intensity(df)
      df = self.create_load_time(df)
      self.write_gold_traffic(df)
      print("Gold traffic loaded")


    def load_gold_roads(self):
      print("Loading gold roads")
      df = self.read_silver_roads()
      df = self.create_load_time(df)
      self.write_gold_roads(df)
      print("Gold roads loaded")
